In [1]:
import re
import json
import os
import sys

parent_path = os.path.abspath('..')
sys.path.append(parent_path)
parent_path = os.path.abspath('../../')
sys.path.append(parent_path)
parent_path = os.path.abspath('../../../')
sys.path.append(parent_path)

from core.ShoreNet.conf import get_data_path

DATA_PATH = get_data_path()

# load .env file
project_path = os.path.abspath('../../')
dotenv_path = os.path.join(project_path, 'secret', '.env')

from dotenv import load_dotenv
load_dotenv(dotenv_path)

print(os.getenv("SISI_DB_TYPE"))

# load project variables
from core.ShoreNet.definitions.variables import VariablesManager

var = VariablesManager()
print(var.data_path)


mysql
D:\data\sisi\


In [2]:
# load dock polygons
from core.ShoreNet.events.dock import get_dock_polygon

dock_polygon_list = get_dock_polygon(var.engine)
print(len(dock_polygon_list))

Dock polygon count: 1518
1518


# 1 Process Sail Events

## 1.1 Cleaning (DBSCAN):

1. Load whole events from SQL database
2. Clean events with DBSCAN, parameter is {radius: 200m, cluster_min_count: 30}
3. Filter noise cluster
4. Further filter: If there are less 20 ships and the count of events less than 60, the cluster will also be recognized as noise cluster.
5. After the double filter, we get the cleaned events dataframe.

## 1.2 Load Statics Data and Events Data

In [3]:
import pandas as pd

from sqlalchemy import text

from core.ShoreNet.events.filter import clean_up_events
from core.ShoreNet.statics.filter import clean_up_statics

# count all sail logging
stop_event_query = f"""
SELECT 
    mmsi, 
    Begin_time as begin_time, End_time as end_time, end_time - begin_time as duration, 
    lng, lat, Point_num as point_num, Event_categories, coal_dock_id
FROM 
    sisi.factor_stop_events t
WHERE
    t.avgSpeed < 1
"""

coal_static_query = f"""
SELECT
    mmsi, shipname, shiptype, length, breadth, DWT_float as dwt
FROM
    sisi.dim_ships_statics
"""

events_df = pd.read_sql(
    sql=text(stop_event_query), con=var.engine
)

statics_df = pd.read_sql(
    sql=text(coal_static_query), con=var.engine
)
statics_df = clean_up_statics(statics_df)

events_df = clean_up_events(
    df=events_df,
    var=var
    # mmsi_enum_list=statics_df['mmsi'].tolist()
)

original event data shape: (1202751, 9)
cleaned event data shape: (108661, 9)


## 1.3 DBSCAN Clustering for Events

In [4]:
from core.ShoreNet.events.dock import cluster_dock_polygon_dbscan

dbscan_res = cluster_dock_polygon_dbscan(
    events_df=events_df,
    var=var
)
print(dbscan_res)

[ 0  0  0 ...  2 -1 -1]
